<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Setup

## Library import
We import all the required Python libraries

The non-default libries are networkX (https://networkx.org/) and py4cytoscape (https://py4cytoscape.readthedocs.io/, only necessary if you wish to view the results in Cytoscape). 

You will also need the skm-tools package provided in the same repository as this notebook. 

In [36]:
#pip install networkx
#pip install --upgrade networkx
# pip install "skm-tools[cytoscape] @ git+https://github.com/NIB-SI/skm-tools.git"

In [37]:
import sys
from pathlib import Path
from datetime import datetime
import networkx as nx
import pandas

In [38]:
from collections import defaultdict

The following allows us to import functions from the skm-tools package. 
Note the relative path to the folder containing the 
"skm_tools" directory. 


In [39]:
# sys.path.append("../")
from skm_tools import load_networks, pss_utils

In [40]:
today = datetime.today().strftime('%Y.%m.%d'); today

'2025.02.26'

## Path and parameter definitions

In [41]:
base_dir = Path("./")
data_dir = base_dir / "../input"
output_dir = base_dir / "../output"
print(data_dir)

..\input


In [42]:
from pathlib import Path

base_dir = Path().resolve()  # Gets the current working directory
data_dir = (base_dir / "../input").resolve()
output_dir = (base_dir / "../output").resolve()

print(data_dir)
print(output_dir)

\\srvljfs.nib.sql\fito\DEJAVNOSTI\OMIKE\pISA-Projects\_p_Endophytes\_I_Bsubtilis\_S_06_networkExpress\_A_03_networkExpress-R\input
\\srvljfs.nib.sql\fito\DEJAVNOSTI\OMIKE\pISA-Projects\_p_Endophytes\_I_Bsubtilis\_S_06_networkExpress\_A_03_networkExpress-R\output


## Load PSS

To obtain the exact results of the article, download PSS-v1.0.0 from [skm.nib.si/downloads](https://skm.nib.si/downloads), and adjust the below paths accordingly. 
Otherwise, this code will use the latest live PSS instance. 


In [43]:
#pss_edge_path = data_dir / f"pss-dinar-edges-restricted_2022-10-26.tsv"
#pss_node_path = data_dir / f"pss-dinar-nodes-restricted_2022-10-26.tsv"
import os

#pss_edge_path = data_dir / f"rxn-edges-public.tsv"
#pss_node_path = data_dir / f"rxn-nodes-public.tsv"

pss_edge_path = data_dir / f"pss-dinar-edges-restricted_2022-10-26_new-format.tsv"
pss_node_path = data_dir / f"pss-dinar-nodes-restricted_2022-10-26_new-format.tsv"

print("Checking files...")
print("Edge file exists:", os.path.exists(pss_edge_path))
print("Node file exists:", os.path.exists(pss_node_path))


Checking files...
Edge file exists: True
Node file exists: True


In [44]:
#for file in data_dir.glob("*"):
#    print(file)

In [45]:
test = pandas.read_table(pss_edge_path)
test.head()
test = pandas.read_table(pss_node_path)
test.head()

,name,node_type,short_name,functional_cluster_id,reaction_id,reaction_type,reaction_effect,additional_information,pathway,all_pathways,...,external_links,family,function,classification,components,ath_homologues,nta_homologues,osa_homologues,stu_homologues,sly_homologues
0,VPg,ForeignCoding,NaN,NaN,NaN,NaN,NaN,NaN,Stress - Biotic,Stress - Biotic,...,doi:10.1016/j.coviro.2012.09.004,NaN,NaN,virus,NaN,NaN,NaN,NaN,NaN,NaN
1,HC-Pro,ForeignCoding,NaN,NaN,NaN,NaN,NaN,papain-like cysteine protease with suppressor ...,Stress - Biotic,Stress - Biotic,...,doi:10.1016/j.coviro.2012.09.004,NaN,NaN,virus,NaN,NaN,NaN,NaN,NaN,NaN
2,CRT,PlantCoding,CRT,fc00048,NaN,NaN,NaN,Non-receptor component required for EFR-mediat...,Signalling - Calcium,Signalling - Calcium,...,gmm:30.3,CRT,NaN,NaN,NaN,"AT1G08450,AT1G09210,AT1G56340",NaN,NaN,NaN,NaN
3,SNRK2,PlantCoding,SNRK2,fc00318,NaN,NaN,NaN,NaN,Hormone - Abscisic acid (ABA),Hormone - Abscisic acid (ABA),...,gmm:29.4.1,NaN,NaN,NaN,NaN,"AT1G10940,AT1G78290,AT4G33950",NaN,NaN,SOTUB02G032470.1.1,NaN
4,ARR-A,PlantCoding,ARR-A,fc00029,NaN,NaN,NaN,type-A negative response regulator,Hormone - Cytokinins (CK),Hormone - Cytokinins (CK),...,gmm:27.3.5,ARR,NaN,NaN,NaN,"AT3G56380,AT3G48100,AT1G19050,AT1G74890,AT3G57...",NaN,NaN,NaN,NaN


In [46]:
g = load_networks.pss_to_networkx(
    edge_path=pss_edge_path, 
    node_path=pss_node_path
)

# print(f"\nNumber of nodes: {g.number_of_nodes()}\nNumber of edges: {g.number_of_edges()}")

## Remove deadend complexes

"deadend" complexes are those created automatically if a binding/oligomerisation is entered into PSS. However, they only make sense to include here if the complex has some downstream interactions it takes part in, since this projection already includes the substrate-substrate interaction. 

The DiNAR projection does not include complexes if the binding/oligomerisation reaction if formulated as inhibition of the substrate(s) (and not activation of the complex). This removes the majority of the problematic complexes. But since (1) the inhibition or activation annotation of the reaction is not always included in the reaction formulation correctlyand (2) somebody maybe hasn't yet added the downstream interactions, let's check for deadend complexes and remove them anyway. 

In [47]:
complexes = [n for n, data in g.nodes(data=True) if data["node_type"] == "Complex"]; complexes

['CML|HC-Pro',
 'EDS1|PAD4',
 'EDS1|MPK3|PAD4',
 'CRT|ETR',
 'HSP90|RAR1|SGT1',
 'EDS1|PAD4|SAG',
 'NDR1|RIN4',
 'NPR1|TGA2,5,6',
 'RANGAP|Rx',
 'GPAphid2|RANGAP',
 'OBE1|WRKY17',
 'OBE1|WRKY11',
 'CO|OBE1',
 'BAK1|FLS2|flg22',
 'MKS1|WRKY33',
 'GA|GID1',
 'WRKY30|WRKY53',
 'SCL14|TGA2,5,6',
 'SCF',
 'GID|SCF',
 'D14|MAX2|SCF',
 'R-gene|potyvirus',
 'EIN3(like)|JAZ',
 'COI1|JA-Ile|SCF',
 'NPR1|NPR1',
 'EBF|SCF',
 'DELLA|GA|GID1',
 'CTR|ETR',
 'ETP|SCF',
 'CML|Ca2+',
 'WD/bHLH/MYB',
 'BAK1|BRI1',
 'SARD1|TCP8',
 'TCP8|WRKY28',
 'NAC019|TCP8',
 'NPR1|WRKY18',
 'PEPR1|PEP1',
 'BIK1|PEPR1|PEP1',
 'ET|ETR']

In [48]:
# g is a directed graph, so we can just use "neighbors" to find complexes without out/downstream edges
to_remove = []
for c in complexes:
    if len(list(g.neighbors(c))) == 0:
        print(c)
        to_remove.append(c)

CML|HC-Pro
EDS1|MPK3|PAD4
CRT|ETR
HSP90|RAR1|SGT1
NDR1|RIN4
RANGAP|Rx
GPAphid2|RANGAP
OBE1|WRKY17
OBE1|WRKY11
CO|OBE1
MKS1|WRKY33
WRKY30|WRKY53
R-gene|potyvirus
EIN3(like)|JAZ
DELLA|GA|GID1
SARD1|TCP8
TCP8|WRKY28
NAC019|TCP8
NPR1|WRKY18
BIK1|PEPR1|PEP1
ET|ETR


In [49]:
g.remove_nodes_from(to_remove)

## filter PSS by types

In [50]:
set([d["node_type"] for n, d in g.nodes(data=True)])

{'Complex',
 'Condition',
 'ForeignAbiotic',
 'ForeignCoding',
 'ForeignEntity',
 'ForeignNonCoding',
 'Metabolite',
 'PlantAbstract',
 'PlantCoding',
 'PlantNonCoding',
 'Process'}

In [51]:
keep = ['Complex',
        'Metabolite',
        'PlantAbstract',
        'PlantCoding', 
        'PlantNonCoding', 
        'Process'
        ]
pss_utils.filter_pss_nodes(g, node_types = keep, remove_isolates=True)

Removed 33 nodes from network.


{'HC-Pro': 'wrong node type',
 'P3': 'wrong node type',
 'VPg': 'wrong node type',
 'CI': 'wrong node type',
 'CP': 'wrong node type',
 'NPR1 high & JAZ high': 'wrong node type',
 'NPR1 high & JAZ low': 'wrong node type',
 'flg22': 'wrong node type',
 'potyvirus': 'wrong node type',
 'Waterlogging': 'wrong node type',
 'SA low': 'wrong node type',
 'NPR1 low & JAZ low': 'wrong node type',
 'trichous-bacteria': 'wrong node type',
 'SA high': 'wrong node type',
 'viral dsRNA': 'wrong node type',
 'vsiRNA': 'wrong node type',
 'me-vsiRNA': 'wrong node type',
 'BAK1|FLS2|flg22': 'complex component removed',
 'EDF2': 'isolate',
 'TDX': 'isolate',
 'RH8': 'isolate',
 'CPIP': 'isolate',
 'DXPS2': 'isolate',
 'ATPB': 'isolate',
 'MIND1': 'isolate',
 'PAA2': 'isolate',
 'PBB2': 'isolate',
 'PBE1': 'isolate',
 'PSAK': 'isolate',
 'O2-deficiency': 'isolate',
 'REM12': 'isolate',
 'EIF4E1': 'isolate',
 'DCL2,4': 'isolate'}

## Remove "uninteresting" metabolites, and connect all their neighbours

In [52]:
metabolites = [n for n, data in g.nodes(data=True) if data["node_type"] == "Metabolite"]; metabolites

['CA',
 'CA-CoA',
 'Ca2+',
 'IsoChor-9-Glu',
 'N-pyruvoyl-L-Glu',
 'SA',
 'GA',
 'DMAPP',
 'prenyl-tRNA',
 'tRNA-adenine',
 'tZ',
 'cZ',
 'DZ',
 'iP',
 'L-Met',
 'SAMe',
 'ACC',
 'ET',
 'Thr',
 'Ile',
 '13-HPOT',
 '12,13-EOT',
 'OPDA',
 'OPC8',
 'OPC8-CoA',
 'OPC6-CoA',
 'OPC4-CoA',
 'JA-CoA',
 'JA',
 'MeJA',
 'JA-Ile',
 '12-OH-JA-Ile',
 'MeSA',
 'Chor',
 'IsoChor',
 'Prep',
 'Phe',
 'BA',
 'SAG',
 'ROS',
 'O2',
 'e-',
 'Geranylgeranyl-PP',
 'ent-Copalyl-PP',
 'ent-Kaurene',
 'ent-Kaurenoic acid',
 'GA12',
 'GA9',
 'GA20',
 'GA8',
 'GA34',
 'GA-methylester',
 '&beta;-Carotene',
 '9-cis-&beta;-carotene',
 'CL',
 'CLA',
 'HMBDP',
 'Cu2+',
 'Violaxanthin',
 '9-cis-Violaxanthin',
 'Phytoene',
 'SL',
 'Abscisic aldehyde',
 'ABA',
 'Xanthoxin',
 'Antheraxanthin',
 'OG',
 'L-arogenate',
 'Tyr',
 'Zeaxanthin',
 '&beta;-Cryptoxanthin',
 '&gamma;-Carotene',
 'Lycopene',
 'all-trans-Neurosporene',
 'all-trans-zeta-Carotene',
 'all-trans-Phytofluene',
 '3H3PP-CoA',
 '3O3PP-CoA',
 '9-cis-10&prime;-

In [53]:
# List here the metabolites to keep as is
#metabolites_to_keep = [
#    'Ca2+',
#    'SA',
#    'GA',
#    'cZ',
#    'ET',
#    'JA',
#    'JA-Ile',
#    'ROS',
#    'O2',
#    'Cu2+',
#    'ABA',
#    'SL',
#    'IAA',
#]
metabolites_to_keep = ['ROS', 'Ca2+']

In [54]:
reaction_types_to_collapse = [
    "catalysis", 
    "translocation"
]

In [55]:
for metabolite in metabolites:
    if not metabolite in metabolites_to_keep:
        
        # find all neighbours that are: (not Metabolite) or (in metabolites_to_keep)
        targets = []
        edges = {}
        sources = []
        
        for target in g.successors(metabolite):
            e = g[metabolite][target][0]
            
            # right type of reaction
            if e['reaction_type'] in reaction_types_to_collapse:

                # right type of target node
                target_type = g.nodes()[target]['node_type']
                if target_type == "Metabolite":
                    if not (target in metabolites_to_keep):
                        continue

                edges[target] = {'interaction_type': 'activation', 
                                'directed': 'True', 
                                'reaction_type': 'catalysis', 
                                'reaction_effect': 'activation', 
                                'reaction_id': e['reaction_id'], 
                                'source_edge_type': 'PRODUCT/SUBSTRATE', 
                                'target_edge_type': 'PRODUCT/SUBSTRATE', 
                                'note': f'metabolite collapse {metabolite}'
                                }
            
                targets.append(g.nodes()[target])

        for source in g.predecessors(metabolite):
            e = g[source][metabolite][0]
            
            # right type of reaction
            if e['reaction_type'] in reaction_types_to_collapse:

                # right type of source node
                source_type = g.nodes()[source]['node_type']
                if source_type == "Metabolite":
                    if not (source in metabolites_to_keep):
                        continue
            
                sources.append(g.nodes()[source])

        for source in sources:
            for target in targets:
                if source != target:
                    g.add_edge(source['name'], target['name'], **edges[target['name']])
        
        g.remove_node(metabolite)

In [56]:
print(f"\nNumber of nodes: {g.number_of_nodes()}\nNumber of edges: {g.number_of_edges()}")


Number of nodes: 253
Number of edges: 442


In [ ]:
# Exercise - remove self loops and duplicated edges?

## main component

In [57]:
g = g.subgraph(max(nx.weakly_connected_components(g), key=len)).copy()

# Cytoscape 

First open the Cytoscape application. Then the following cell will load the required library and and make sure you can connect to the Cytoscape application. 

More py4cytoscape documentation is here: https://py4cytoscape.readthedocs.io/

In [58]:
from skm_tools import cytoscape_utils

## Start Cytoscape and then proceed

In [59]:
import py4cytoscape as p4c
p4c.cytoscape_ping()
p4c.cytoscape_version_info()

You are connected to Cytoscape!


{'apiVersion': 'v1',
 'cytoscapeVersion': '3.9.1',
 'automationAPIVersion': '1.11.0',
 'py4cytoscapeVersion': '1.11.0'}

We set the Cytoscape collection name for this notebook. 

In [60]:
COLLECTION = f"PSS DiNAR projection ({today})"
COLLECTION

'PSS DiNAR projection (2025.02.26)'

## Load the PSS network into Cytoscape

We load the network, set a visual style, and apply the CoSE layout.

With skm-tools, we provide a default style for PSS, colouring the nodes by pathway.

Returned is the ID of the network view in Cytoscape.

In [61]:
pss_network_suid = p4c.networks.create_network_from_networkx(g, title="Complete PSS", collection=COLLECTION)

Applying default style...
Applying preferred layout


## Exporting graph to file

In [62]:
df = nx.to_pandas_edgelist(g)
df.head()
df.to_csv(output_dir / f"pss-refined-edgelist-{today}.tsv", sep="\t", index=None)

## coordinates

In [ ]:
# https://gist.github.com/carissableker/620f10ffdaedcb18faeac52aa1a9acdb

# ignore all below

In [ ]:
# Apply a style
#cytoscape_utils.apply_builtin_style(pss_network_suid, 'pss')
#pss_network_suid

# Example path extraction

Before running this section, depending on the problem at hand, you may want to remove nodes such as virus related, of perhaps abiotic stress. 


First we identify the nodes of interest.

In [ ]:
JA = [x for x,y in g.nodes(data=True) if y['name']=="JA"][0]
SA = [x for x,y in g.nodes(data=True) if y['name']=="SA"][0];
ROS = [x for x,y in g.nodes(data=True) if y['name']=="ROS"][0]

print(JA)
print(SA)
print(ROS)

## JA --> SA + JA --> ROS

In [ ]:
JA_paths = [p for p in nx.all_shortest_paths(g, source=JA, target=SA)]
JA_paths += [p for p in nx.all_shortest_paths(g, source=JA, target=ROS)]
JA_paths

## SA --> JA + SA --> ROS

In [ ]:
SA_paths = [p for p in nx.all_shortest_paths(g, source=SA, target=JA)]
SA_paths += [p for p in nx.all_shortest_paths(g, source=SA, target=ROS)]
SA_paths

## ROS --> JA + ROS --> SA

In [ ]:
ROS_paths = [p for p in nx.all_shortest_paths(g, source=ROS, target=JA)]
ROS_paths += [p for p in nx.all_shortest_paths(g, source=ROS, target=SA)]
ROS_paths

# Visualise paths in Cytoscape

Now we're going to highlight the paths we identified in the network by applying style bypasses.



We set the colours here:

In [ ]:
JA_COLOUR = "#66a61e"
SA_COLOUR = "#34858d"
ROS_COLOUR = "#dc1c1c"

We don't want to recolour already highlighted path elements, so we keep track of them here:

In [ ]:
done_nodes, done_edges = [], []

In [ ]:
for p in ROS_paths:
    done_nodes_now, done_edges_now = cytoscape_utils.highlight_path(p, ROS_COLOUR, skip_nodes=done_nodes, skip_edges=done_edges)
    done_nodes += done_nodes_now
    done_edges += done_edges_now
# Note - need to improve edge colouring

In [ ]:
for p in JA_paths:
    done_nodes_now, done_edges_now = cytoscape_utils.highlight_path(p, JA_COLOUR, skip_nodes=done_nodes, skip_edges=done_edges)
    done_nodes += done_nodes_now
    done_edges += done_edges_now

In [ ]:
for p in SA_paths:
    print(p)
    done_nodes_now, done_edges_now = cytoscape_utils.highlight_path(p, SA_COLOUR, skip_nodes=done_nodes, skip_edges=done_edges)
    done_nodes += done_nodes_now
    done_edges += done_edges_now


At this point, the Cytoscape session has a network view of the filtered PSS, and highlighting of the paths we extracted from our targeted searches. 

## Extract subnetworks in Cytoscape

Properly inspecting the identified paths is a bit hard within the complete network, so here we pull out the subnetworks of and surrounding the paths. 


### The edge induced network
The first, and smallest subnetwork, is created by extracting only the edges that are present on the paths. 

In [ ]:
network_edge_induced_suid = cytoscape_utils.subnetwork_edge_induced_from_paths(
    paths=JA_paths + SA_paths + ROS_paths,
    g=g,
    parent_suid=pss_network_suid,
    name="identified paths (edge induced)",
)

We apply a new layout to this subnetwork

In [ ]:
_ = p4c.layouts.layout_network('cose', network=network_edge_induced_suid)

### The node induced network

Now we extract the network based on the nodes along the paths, meaning any edges between those nodes that are not on the paths are also extracted. 

In [ ]:
nodes = list(set([y for x in JA_paths + SA_paths + ROS_paths for y in x]))

In [ ]:
network_node_induced_suid = cytoscape_utils.subnetwork_node_induced(
    nodes=nodes,
    parent_suid=pss_network_suid,
    name="identified paths (node induced)",
)

Instead of applying a network layout algorithm, we can copy the layout from the previous subnetwork. 

In [ ]:
_ = p4c.layouts.layout_copycat(
    network_edge_induced_suid, 
    network_node_induced_suid
)

### Neighbours

For more context around our paths, we can include the first neighbours in the view. We can use the Cytoscape first neighbour selection functionality. 

In [ ]:
network_neighbours_suid = cytoscape_utils.subnetwork_neighbours(
    nodes=nodes,
    parent_suid=pss_network_suid,
    name="identified paths + 1st neighbours",
)

In [ ]:
_ = p4c.layouts.layout_network('cose', network=network_neighbours_suid)

### Additional filtering of the neighbours

There are many neighbours displayed now, and we are perhaps only interested in the ones that are connected to at least two of the original path nodes, so we can make a filter using networkX neighbour functions. 

In [ ]:
filtered_neighbours = []
for n in g.nodes():
    if (len([x for x in nx.MultiGraph(g).neighbors(n) if (x in done_nodes)]) > 1) and (n not in done_nodes):
        filtered_neighbours.append(n)

In [ ]:
network_neighbours_filtered_suid = cytoscape_utils.subnetwork_node_induced(
    nodes=nodes+filtered_neighbours,
    parent_suid=pss_network_suid,
    name="identified paths + 1st neighbours (filtered)",
)

In [ ]:
p4c.layouts.layout_copycat(
    network_neighbours_suid, 
    network_neighbours_filtered_suid
)

## Saving the session

Save the Cytoscape session:

In [ ]:
p4c.session.save_session(str(output_dir / f"PSS-DiNAR-JA-SA-ROS-{today}.cys"))

## Exporting graph to file

NetworkX does not have  the greatest export formats, so  making a Pandas dataframe and saving that seems the best. 

In [ ]:
df = nx.to_pandas_edgelist(g)
df.head()

In [ ]:
df.to_csv(output_dir / f"pss-dinar-refined-edgelist-{today}.tsv", sep="\t", index=None)

### Exporting the subnetwork

In [ ]:
g_subgraph_node_induced = nx.induced_subgraph(g, nodes)
print(f"\nNumber of nodes: {g_subgraph_node_induced.number_of_nodes()}\nNumber of edges: {g_subgraph_node_induced.number_of_edges()}")

In [ ]:
nx.to_pandas_edgelist(g_subgraph_node_induced)\
    .to_csv(output_dir / f"pss-dinar-subgraph-refined-edgelist-{today}.tsv", sep="\t", index=None)

## Exporting the Cytoscape networks to PDF

In [ ]:
collection_suid = p4c.get_collection_suid(network_edge_induced_suid)

In [ ]:
from skm_tools import cytoscape_pdf_utils

In [ ]:
cytoscape_pdf_utils.export_collection_to_pdfs(collection_suid, output_dir / "figures")

In [ ]:
cytoscape_pdf_utils.export_collection_to_single_pdf(collection_suid, output_dir / "figures" / "single_pdf", caption=True)

In [ ]:
# END